#Linea de espera

$$G/G/1$$

In [69]:
from random import random
from random import expovariate as expo
from random import triangular
import numpy as np
import pandas as pd

class Contar:
	def __init__(self,cota):
		self.cota = cota
	def c(self,x):
		if x < self.cota:
			return x

def lineaE(n):
	t0 = 0
	s = 0 # total de clientes en la fila
	k = 0 # indice
	datos = np.zeros( (n,8) )
	encabezados = ['cliente',
	               'tiempo de interllegada',
									'tiempo de llegada',
									'fila',
									'inicio del servicio',
									'tiempo de servicio',
									'fin del servicio','tiempo en el sistema']
	for i in range(n):
		# numero de cliente
		datos[i][0] = i + 1

		# tiempo entre llegadas
		t = expo(1/15) # distribucion de llegadas
		datos[i][1] = t

		# tiempo de llegada
		if i == 0:
			t0 = t
			datos[i][2] = t0
		else:
			t0 += t
			datos[i][2] = t0

		# fila
		if i == 0:
			datos[i][3] = s
		elif t0 < datos[k][6]:
			s += 1
			datos[i][3] = s
		elif t0 > datos[k][6]:
			w = len( list( filter( Contar( datos[i][2] ).c , datos[k:i,6] )))
			s = len( datos[k:i,6] ) - w
			k += w # se actualiza el indice
			datos[i][3] = s

		# inicio del servicio
		if i==0:
			datos[i][4] = t
		else:
			datos[i][4] = max( datos[i-1][6] , t0 )

		# distribucion del tiempo de servicio
		u = expo(1/8)
		datos[i][5] = u

		# fin del servicio
		datos[i][6] = datos[i][4] + u

		# tiempo en el sistema
		datos[i][7] = datos[i][6] - t0
	return pd.DataFrame( np.round(datos,2) , columns = encabezados,dtype=float).astype({'cliente':np.int32,
'tiempo de interllegada':np.float32,
'tiempo de llegada':np.float32,
'fila':np.int32,
'inicio del servicio':np.float32,
'tiempo de servicio':np.float32,
'fin del servicio':np.float32,
'tiempo en el sistema':np.float32})

simu = lineaE(60).style.hide( axis = "index" ).format( precision = 2 )
simu

cliente,tiempo de interllegada,tiempo de llegada,fila,inicio del servicio,tiempo de servicio,fin del servicio,tiempo en el sistema
1,44.47,44.47,0,44.47,21.31,65.78,21.31
2,8.41,52.88,1,65.78,9.32,75.09,22.21
3,16.84,69.72,1,75.09,3.72,78.81,9.09
4,0.58,70.30,2,78.81,5.25,84.06,13.76
5,11.98,82.29,1,84.06,7.20,91.26,8.97
6,8.33,90.61,1,91.26,0.65,91.91,1.29
7,5.35,95.96,0,95.96,3.73,99.69,3.73
8,3.22,99.18,1,99.69,6.21,105.90,6.72
9,9.27,108.45,0,108.45,2.07,110.52,2.07
10,1.06,109.51,1,110.52,1.01,111.53,2.02


In [70]:
k = 10000
simu2 = lineaE(k)
print("Clientes promedio en la fila: ",simu2['fila'].mean())
print("Probabilidad de cero clientes en el sistema:",list( simu2["fila"] ).count(0)/k)

Clientes promedio en la fila:  1.0727
Probabilidad de cero clientes en el sistema: 0.4728
